In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
from astropy import units
from pathlib import Path
import os

import snapshot_obj
import dataset_compute
import curve_fit

import importlib

In [ ]:
importlib.reload(snapshot_obj)
importlib.reload(dataset_compute)
importlib.reload(curve_fit)

# Differences between values of $v_\mathrm{max}$ 

To validate the values computed with my implemented methods, I compare them to the catalogued values that should be alright.

## Inspect subhalo sizes

First let us plot a histogram (logarithmically) of the number of bound particles of the subhalos in the med-res run. As expected, most halos are formed in the small-mass range, with only about $<100$ bound particles. These halos are hardly resolved.

In [ ]:
# Get LCDM data:
LCDM = snapshot_obj.Snapshot("V1_MR_fix", 127)
vmax = LCDM.get_subhalos("Vmax") * units.cm.to(units.km)
data = LCDM.get_subhalos("Max_Vcirc")
max_vcirc = data[:,0] * units.cm.to(units.km)
rmax = data[:,1] * units.cm.to(units.kpc)

In [ ]:
part_nums = LCDM.get_subhalos("SubLength")
print(np.min(part_nums))

In [ ]:
down = 1; up = 5
part_nums = np.log10(part_nums)
part_nums = part_nums[np.logical_and(down < part_nums, part_nums < up)]
fig, axes = plt.subplots()
axes.set_xlabel('$\mathrm{log}(N_\mathrm{bound})$')
axes.hist(part_nums, 10)
plt.show()

## Plot differing values

Now, let us find the subhalos, for which the difference in the $v_\textrm{max}$ value is in a given range. We see that subhalos with a greater disagreement are the subhalos with less particles. The lines that form in the small-mass range of the figure correspond to $n_\mathrm{soft}, n_\mathrm{soft}*2, n_\mathrm{soft}*3, \dots$ dm particles. The value of $n_\mathrm{soft}$ only makes a difference very close to the halo centre (where the accumulated number of particles is small). However, to have a meaningful value of $v_\mathrm{max}$ for the subhalos with around 50 particles, I chose $n_\mathrm{soft}=5$ (instead of 10 used for rotation curves). We certainly should not be concerned with the subhalos with less than 50 particles (and talking about halos this small is already pushing it a bit).

Looking at subhalos with 5% difference in the values, all of them are among the subhalos that are not really resolved yet. Being able to compute $v_\mathrm{max}$ for the interesting halos with 5% accuracy, in the very least, is sufficient-

In [ ]:
err = np.abs(vmax-max_vcirc)/vmax
differing = np.logical_and(0.01 < err, err < 1)
print(vmax.size)
print(np.sum(differing))

In [ ]:
fig, axes = plt.subplots()

# Set axis:
axes.set_xscale('log')
axes.set_yscale('log')
axes.set_xlim(0.1, 50)
axes.set_ylim(0.3, 100)

# Add LCDM scatter plot:
axes.scatter(rmax[differing], vmax[differing], s=3, label='catalogue')
axes.scatter(rmax[differing], max_vcirc[differing], s=3, label='me')

plt.legend()
plt.tight_layout()
